In [1]:
import spacy
import scispacy
import swifter
import pandas as pd
from spacy import displacy
# import en_core_sci_sm
import en_ner_bc5cdr_md
# import en_ner_jnlpba_md
# import en_ner_craft_md
# import en_ner_bionlp13cg_md
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from collections import OrderedDict,Counter
from pprint import pprint
from tqdm import tqdm
import os
tqdm.pandas()

/gpfs/projects/bsc14/code/MultiSynDS/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


Not capable of splitting into sentences, but can be split into words.

In [ ]:
import scispacy
import spacy

file_orig = "data/1_original/txt/25350173.txt"
file_gen = "data/2_generated/2step_transformation_dt4h_GPT4omini/en/25350173_transformed_step1.txt"
ner_model = "en_core_sci_sm"

with open(file_orig, "r") as f_orig:
    text_orig = f_orig.read()
    
with open(file_gen, "r") as f_gen:
    text_gen = f_gen.read()

nlp = en_ner_bc5cdr_md.load()

# nlp = spacy.load(ner_model)

try:
    @spacy.Language.factory("abv_detector")
    def create_abv_detector(nlp, name):
        return AbbreviationDetector(nlp)

    nlp.add_pipe("abv_detector")
except ValueError:
    print("Abbreviation detector already exists")
    
# Entitiy linking

from spacy.language import Language
# import en_core_sci_sm
from scispacy.linking import EntityLinker

# nlp = en_core_sci_sm.load()

try:    # Register the EntityLinker component
    @Language.factory("umls_linker")
    def create_umls_linker(nlp, name):
        return EntityLinker(k=10, max_entities_per_mention=5, name="umls")

    nlp.add_pipe("umls_linker")
except ValueError:
    print("Entity linker already exists")

/gpfs/projects/bsc14/code/MultiSynDS/.venv/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/gpfs/projects/bsc14/code/MultiSynDS/.venv/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
doc_orig = nlp(text_orig)
doc_gen = nlp(text_gen)

In [4]:
orig_ent = [(ent.text.lower(), ent.label_) for ent in doc_orig.ents] + [(ent._.long_form.text.lower(), ent.label_) for ent in doc_orig.ents if ent._.long_form is not None]
gen_ent = [(ent.text.lower(), ent.label_) for ent in doc_gen.ents] + [(ent._.long_form.text.lower(), ent.label_) for ent in doc_gen.ents if ent in doc_gen._.abbreviations]

In [5]:
print("Original entities")
print(orig_ent)
print("\nGenerated entities")
print(gen_ent)

Original entities
[('dyspnoea', 'DISEASE'), ('ventricular tachycardia', 'DISEASE'), ('amiodarone', 'CHEMICAL'), ('adenosine', 'CHEMICAL'), ('aneurysm', 'DISEASE'), ('arrhythmia', 'DISEASE'), ('lv stroke', 'DISEASE')]

Generated entities
[('dyspnoea', 'DISEASE'), ('ventricular tachycardia', 'DISEASE'), ('vt', 'DISEASE'), ('aneurysm', 'DISEASE'), ('amiodarone', 'CHEMICAL'), ('adenosine', 'CHEMICAL'), ('ventricular remodelling', 'DISEASE'), ('lv stroke', 'DISEASE')]


In [6]:
print(set(gen_ent) - set(orig_ent))
print(set(orig_ent) - set(gen_ent))

{('ventricular remodelling', 'DISEASE'), ('vt', 'DISEASE')}
{('arrhythmia', 'DISEASE')}


In [7]:
linker = nlp.get_pipe("umls_linker")

umls_gen = [(ent, ent.label_, [cui[0] for cui in ent._.kb_ents], [linker.kb.cui_to_entity[cui[0]].canonical_name for cui in ent._.kb_ents]) for ent in doc_gen.ents]
umls_gen

[(Dyspnoea,
  'DISEASE',
  ['C0013404', 'C0743330', 'C0231807'],
  ['Dyspnea', 'Resting Dyspnea', 'Dyspnea on exertion']),
 (ventricular tachycardia,
  'DISEASE',
  ['C0042514', 'C0344428'],
  ['Tachycardia, Ventricular', 'Ventricular Tachycardia by ECG Finding']),
 (VT,
  'DISEASE',
  ['C0042514', 'C0344428'],
  ['Tachycardia, Ventricular', 'Ventricular Tachycardia by ECG Finding']),
 (aneurysm,
  'DISEASE',
  ['C0002940', 'C0751003'],
  ['Aneurysm', 'Brain Aneurysm']),
 (amiodarone, 'CHEMICAL', ['C0002598'], ['amiodarone']),
 (adenosine,
  'CHEMICAL',
  ['C0001443', 'C1158747', 'C0001464'],
  ['adenosine', 'adenosine to inosine editing', 'Adenosine Kinase']),
 (ventricular remodelling,
  'DISEASE',
  ['C0600519', 'C4761623'],
  ['Ventricular Remodeling', 'Left Ventricular Remodeling (procedure)']),
 (LV stroke, 'DISEASE', ['C0038454'], ['Cerebrovascular accident'])]

In [8]:
def entity_linker(nlp, text, linker_name="umls_linker"):
    doc = nlp(text)
    linker = nlp.get_pipe(linker_name)
    return [(ent.text, ent.label_, ent._.kb_ents[0][0] if ent._.kb_ents != [] else None, linker.kb.cui_to_entity[ent._.kb_ents[0][0]].canonical_name if ent._.kb_ents != [] else None) for ent in doc.ents]

In [ ]:
import pandas as pd
import os

# import matplotlib.pyplot as plt

main_path = "data/2_generated/2step_transformation_dt4h_GPT4omini"
lang = "en"
main_path = os.path.join(main_path, lang)

In [10]:
def extract_txt(path, filename):
    total_path = os.path.join(path, filename)
    return filename, open(total_path, "r").read()

def files_to_df(path, extensions=["txt"]):
    files = [x for x in os.listdir(path) if x.split(".")[-1] in extensions]
    data = [extract_txt(path, f) for f in files]
    return pd.DataFrame(data, columns=["filenameid", "text"])

df = files_to_df(main_path)
df["text_orig"] = df["text"]
df["text"] = df["text_orig"].apply(lambda x: x.split("'text_to_transform': ")[-1][:-1].replace("'", ""))
df.drop("text_orig", axis=1, inplace=True)
df

,filenameid,text
0,36951253_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
1,33892935_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
2,28960930_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
3,29538200_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
4,30103814_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
...,...,...
995,30762762_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
996,27980283_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
997,34012203_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
998,37828735_4_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...


In [11]:
df_ents = pd.DataFrame()
step = 100
for i in range(0, df.shape[0], step):
    df_ents = pd.concat([df_ents, df.iloc[i:i+step].set_index("filenameid")["text"].swifter.apply(lambda x: entity_linker(nlp, x)).explode().apply(pd.Series)], axis=0)
    
# df_ents = df.set_index("filenameid")["text"].swifter.apply(lambda x: entity_linker(nlp, x)).explode().apply(pd.Series)
df_ents.columns = ["span", "mention_class", "code", "term"]
df_ents.reset_index(inplace=True)

Pandas Apply:   3%|▎         | 3/100 [00:00<00:05, 16.27it/s]/gpfs/projects/bsc14/code/MultiSynDS/.venv/lib64/python3.9/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
Pandas Apply:   3%|▎         | 3/100 [00:00<00:04, 24.08it/s]/gpfs/projects/bsc14/code/MultiSynDS/.venv/lib64/python3.9/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]/gpfs/projects/bsc14/code/MultiSynDS/.venv/lib64/python3.9/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]/gpfs/projects/bsc14/code/MultiSynDS/.venv/lib64/python3.9/site-packages/scispacy/abbreviat

In [12]:
df_ents

,filenameid,span,mention_class,code,term
0,36951253_transformed_step1.txt,End-stage dilated non-ischaemic cardiomyopathy,DISEASE,None,None
1,36951253_transformed_step1.txt,myocarditis,DISEASE,C0027059,Myocarditis
2,36951253_transformed_step1.txt,sarcopenia,DISEASE,C0872084,Sarcopenia
3,36951253_transformed_step1.txt,cardiac cachexia,DISEASE,C0562491,Cardiac cachexia
4,36951253_transformed_step1.txt,cardiorespiratory arrest,DISEASE,C0600228,Cardiopulmonary Arrest
...,...,...,...,...,...
25279,33045427_transformed_step1.txt,cardiotropic infectious,DISEASE,None,None
25280,33045427_transformed_step1.txt,post-EMB,CHEMICAL,None,None
25281,33045427_transformed_step1.txt,CD45R0,CHEMICAL,C3891563,"CD45RO, human"
25282,33045427_transformed_step1.txt,CD11a+,CHEMICAL,C0079091,CD11a Antigen
